In [1]:
import argparse
import os
import torch
import numpy as np
import random
import pandas as pd


from exp.exp_NRU_RBN import Exp_NRU_RBN
%matplotlib inline

parser = argparse.ArgumentParser(description='NRU_RBN Forecasting')

parser.add_argument('--model', type=str, default='NRU_RBN',help='model of experiment, options: [NRU_RBN]')
value="ETTh1" #ETTh1,ETTh2,electrans,weather,exchange,BTC
parser.add_argument('--data', type=str, default=value, help='data')
parser.add_argument('--root_path', type=str, default='datasets', help='root path of the data file')
parser.add_argument('--data_path', type=str, default=value+'.csv', help='data file')    #electrans.csv
parser.add_argument('--features', type=str, default='MS', help='forecasting task, options:[M, S, MS]; M:multivariate predict multivariate, S:univariate predict univariate, MS:multivariate predict univariate')
parser.add_argument('--target', type=str, default='OT', help='target feature in S or MS task')
parser.add_argument('--freq', type=str, default='h', help='freq for time features encoding, options:[s:secondly, t:minutely, h:hourly, d:daily, b:business days, w:weekly, m:monthly], you can also use more detailed freq like 15min or 3h')
parser.add_argument('--checkpoints', type=str, default= 'model_parameters/'+value, help='location of model checkpoints')

parser.add_argument('--seq_len', type=int, default=(90)+1, help='input sequence length of Informer encoder')
parser.add_argument('--label_len', type=int, default=90, help='start token length of Informer decoder')
parser.add_argument('--pred_len', type=int, default=30, help='prediction sequence length')
# Informer decoder input: concat[start token series(label_len), zero padding series(pred_len)]

parser.add_argument('--enc_in', type=int, default=3, help='encoder input size')
parser.add_argument('--dec_in', type=int, default=3, help='decoder input size')
parser.add_argument('--c_out', type=int, default=3, help='output size')
parser.add_argument('--d_model', type=int, default=256, help='dimension of model')
parser.add_argument('--d_ff', type=int, default=256, help='dimension of fcn')
parser.add_argument('--padding', type=int, default=0, help='padding type')
parser.add_argument('--distil', action='store_false', help='whether to use distilling in encoder, using this argument means not using distilling', default=False)
parser.add_argument('--dropout', type=float, default=0.05, help='dropout')
parser.add_argument('--embed', type=str, default='timeF', help='time features encoding, options:[timeF, fixed, learned]')
parser.add_argument('--activation', type=str, default='gelu',help='activation')
parser.add_argument('--output_attention', action='store_true', help='whether to output attention in ecoder',default=False)
parser.add_argument('--do_predict', action='store_false', help='whether to predict unseen future data')
parser.add_argument('--mix', action='store_false', help='use mix attention in generative decoder', default=True)
parser.add_argument('--cols', type=str, nargs='+', help='certain cols from the data files as the input features')
parser.add_argument('--num_workers', type=int, default=32, help='data loader num workers')
parser.add_argument('--itr', type=int, default=3, help='experiments times')
parser.add_argument('--train_epochs', type=int, default=20, help='train epochs')
parser.add_argument('--batch_size', type=int, default=128, help='batch size of train input data')
parser.add_argument('--patience', type=int, default=2, help='early stopping patience')
parser.add_argument('--learning_rate', type=float, default=0.0001, help='optimizer learning rate')
parser.add_argument('--des', type=str, default='test',help='exp description')
parser.add_argument('--loss', type=str, default='mse',help='loss function')
parser.add_argument('--lradj', type=str, default='type1',help='adjust learning rate')
parser.add_argument('--use_amp', action='store_true', help='use automatic mixed precision training', default=False)
parser.add_argument('--inverse', action='store_true', help='inverse output data', default=False)
parser.add_argument('--likeloss', action='store_true', help='likeloss', default=True)
parser.add_argument('--use_gpu', type=bool, default=True, help='use gpu')
parser.add_argument('--gpu', type=int, default=0, help='gpu')
parser.add_argument('--use_multi_gpu', action='store_true', help='use multiple gpus', default=False)
parser.add_argument('--devices', type=str, default='0,1,2,3',help='device ids of multile gpus')
parser.add_argument('--lossregularizer', type=bool, default=True, help='Use loss regularizer')

args = parser.parse_args("")
args.use_gpu = True if torch.cuda.is_available() and args.use_gpu else False

if args.use_gpu and args.use_multi_gpu:
    args.devices = args.devices.replace(' ','')
    device_ids = args.devices.split(',')
    args.device_ids = [int(id_) for id_ in device_ids]
    args.gpu = args.device_ids[0]

data_parser = {
    'exchange':{'data':'exchange.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'d'},
    'ETTh1':{'data':'ETTh1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'ETTh2':{'data':'ETTh2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'ETTm1':{'data':'ETTm1.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'ETTm2':{'data':'ETTm2.csv','T':'OT','M':[7,7,7],'S':[1,1,1],'MS':[7,7,1],'freqin':'h'},
    'WTH':{'data':'WTH.csv','T':'WetBulbCelsius','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1],'freqin':'h'},
    'weather':{'data':'weather.csv','T':'WetBulbCelsius','M':[12,12,12],'S':[1,1,1],'MS':[12,12,1],'freqin':'h'},
    'electrans':{'data':'electrans.csv','T':'MT_24','M':[316,316,316],'S':[1,1,1],'MS':[316,316,1],'freqin':'h'},
}
if args.data in data_parser.keys():
    data_info = data_parser[args.data]
    args.data_path = data_info['data']
    args.target = data_info['T']
    args.enc_in, args.dec_in, args.c_out = data_info[args.features]
    args.freq=data_info['freqin']

args.detail_freq = args.freq
args.freq = args.freq[-1:]
args.likeloss =args.lossregularizer
print('Args in experiment:')
print(args)
torch.cuda.empty_cache()
torch.manual_seed(0)
np.random.seed(0)
random.seed(0)

Args in experiment:
Namespace(model='NRU_RBN', data='ETTh1', root_path='datasets', data_path='ETTh1.csv', features='MS', target='OT', freq='h', checkpoints='model_parameters/ETTh1', seq_len=91, label_len=90, pred_len=30, enc_in=7, dec_in=7, c_out=1, d_model=256, d_ff=256, padding=0, distil=False, dropout=0.05, embed='timeF', activation='gelu', output_attention=False, do_predict=True, mix=True, cols=None, num_workers=32, itr=3, train_epochs=20, batch_size=128, patience=2, learning_rate=0.0001, des='test', loss='mse', lradj='type1', use_amp=False, inverse=False, likeloss=True, use_gpu=True, gpu=0, use_multi_gpu=False, devices='0,1,2,3', lossregularizer=True, detail_freq='h')


In [2]:
#test&train
torch.cuda.empty_cache()

mape_list=[]

mae_list=[]
rmse_list=[]
dtw_list=[]
res_list=[]
shape_list=[]
corr_list=[]
crps_list=[]
mse_list=[]
crps_varlist=[]

Exp = Exp_NRU_RBN
def runtrain(index,args):
    for ii in range(args.itr):
            # setting record of experiments
            setting = 'id{}_m{}_d{}_ft{}_sl{}_ll{}_pl{}_dm{}_df{}_eb{}_{}_lossreg_{}'.format(
                    index,args.model, args.data, args.features,args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.d_ff, 
                    args.embed, 0,str(args.lossregularizer))

            exp = Exp(args) # set experiments
            print('>>>>>>>start training : {}>>>>>>>>>>>>>>>>>>>>>>>>>>'.format(setting))
        
            exp.train(setting)
            print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
            
            resultout,_ ,_,crpsret=exp.test(setting,False)
            mae, mse, rmse, mape,_, dtw,corr,shapeval =resultout
            attns=0
            mse_list.append(mse)
            mse_np=np.array(mse_list)
            mae_list.append(mae)
            mae_np=np.array(mae_list)
            crps_list.append(np.mean(crpsret))
            crps_varlist.append(np.var(crpsret))
            crps_np=np.array(crps_list)
            crps_varlist_np=np.array(crps_varlist)
            print("Index:"+str(index)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.mean(crps_np))+",var:"+str(np.var(crps_varlist_np)))
            print("Index:"+str(index)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.mean(mse_np))+",var:"+str(np.var(mse_np)))
            print("Index:"+str(index)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.mean(mae_np))+",var:"+str(np.var(mae_np)))
            
            torch.cuda.empty_cache()

if args.data_path == 'electrans.csv' or args.data_path == 'exchange.csv':
        df_raw = pd.read_csv(os.path.join(args.root_path,
                                          args.data+"_all.csv")).set_index('date')
        t,c=df_raw.shape
        for index in range(c):
                print("Index: ", index)
                df_use = df_raw.iloc[:,index]
                path = "datasets/"+args.data+".csv"
                df_use.to_csv(path)
                runtrain(index,args)
else:
        runtrain(0,args)

Use GPU: cuda:0
Num_params : 3432394 
>>>>>>>start training : id0_mNRU_RBN_dETTh1_ftMS_sl91_ll90_pl30_dm256_df256_ebtimeF_0_lossreg_True>>>>>>>>>>>>>>>>>>>>>>>>>>
train 12074
val 2493
test 2493
No File, Train new


100%|██████████| 94/94 [00:50<00:00,  1.86it/s]


Epoch: 1 cost time: 50.65706419944763


100%|██████████| 19/19 [00:05<00:00,  3.44it/s]


vali_CRPS_mean:0.4144927 vali_CRPS_var:0.00033199313


100%|██████████| 19/19 [00:05<00:00,  3.39it/s]


test_CRPS_mean:0.24613762 test_CRPS_var:0.010629237
Epoch: 1, Steps: 94 | Train Loss: 0.8166580 Vali Loss: 0.7225042 Test Loss: 0.7980182
Validation loss decreased (inf --> 0.722504).  Saving model ...


RuntimeError: Parent directory model_parameters/ETTh1 does not exist.

In [ ]:
#
#test&train


mape_list=[]

mae_list=[]
rmse_list=[]
dtw_list=[]
res_list=[]
shape_list=[]
corr_list=[]
crps_list=[]
mse_list=[]
crps_varlist=[]

Exp = Exp_NRU_RBN
def runtrain(index,args):
    
    setting = 'id{}_m{}_d{}_ft{}_sl{}_ll{}_pl{}_dm{}_df{}_eb{}_{}_lossreg_{}'.format(
                    index,args.model, args.data, args.features,args.seq_len, args.label_len, args.pred_len,
                    args.d_model, args.d_ff, 
                    args.embed, 0,str(args.lossregularizer))exp = Exp(args)
    print('>>>>>>>testing : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
    resultout,_ ,_,crpsret=exp.test(setting,True)
    mae, mse, rmse, mape,_, dtw,corr,shapeval =resultout
    attns=0
    """if args.do_predict:
        print('>>>>>>>predicting : {}<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<'.format(setting))
        exp.predict(setting, True)"""

    torch.cuda.empty_cache()
    return crpsret,mae, mse, rmse, mape,_, dtw,corr,shapeval


if args.data_path == 'electrans.csv'or args.data_path == 'exchange.csv':
        df_raw = pd.read_csv(os.path.join(args.root_path,
                                          args.data+"_all.csv")).set_index('date')
        t,c=df_raw.shape
        for index in range(0, c):
                print("Index: ", index)
                df_use = df_raw.iloc[:,index]
                path = "datasets/"+args.data+".csv"
                df_use.to_csv(path)
                crpsret,mae, mse, rmse, mape,_, dtw,corr,shapeval=runtrain(index,args)
                mse_list.append(mse)
                mse_np=np.array(mse_list)
                mae_list.append(mae)
                mae_np=np.array(mae_list)
                crps_list.append(np.mean(crpsret))
                crps_varlist.append(np.var(crpsret))
                crps_np=np.array(crps_list)
                crps_varlist_np=np.array(crps_varlist)
                print("Index:"+str(index)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.mean(crps_np))+",var:"+str(np.var(crps_varlist_np)))
                print("Index:"+str(index)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.mean(mse_np))+",var:"+str(np.var(mse_np)))
                print("Index:"+str(index)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.mean(mae_np))+",var:"+str(np.var(mae_np)))

else:
        crpsret,mae, mse, rmse, mape,_, dtw,corr,shapeval=runtrain(0,args)
        mse_list.append(mse)
        mse_np=np.array(mse_list)
        mae_list.append(mae)
        mae_np=np.array(mae_list)
        crps_list.append(np.mean(crpsret))
        crps_varlist.append(np.var(crpsret))
        crps_np=np.array(crps_list)
        crps_varlist_np=np.array(crps_varlist)
        print("Index:"+str(0)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.mean(crps_np))+",var:"+str(np.var(crps_varlist_np)))
        print("Index:"+str(0)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.mean(mse_np))+",var:"+str(np.var(mse_np)))
        print("Index:"+str(0)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.mean(mae_np))+",var:"+str(np.var(mae_np)))




NameError: name 'Exp_NRU_RBN' is not defined

In [ ]:
print("Index:"+str(0)+",CRPS_L:"+str(len(crps_np))+",AVE:"+str(np.mean(crps_np))+",var:"+str(np.var(crps_varlist_np)))
print("Index:"+str(0)+",MSE_L:"+str(len(mse_np))+",AVE:"+str(np.mean(mse_np))+",var:"+str(np.var(mse_np)))
print("Index:"+str(0)+",MAE_L:"+str(len(mae_np))+",AVE:"+str(np.mean(mae_np))+",var:"+str(np.var(mae_np)))


Index:0,CRPS_L:321,AVE:0.17263213,var:0.00015839086
Index:0,MSE_L:321,AVE:0.13705546,var:0.020117808
Index:0,MAE_L:321,AVE:0.16893287,var:0.011108408


In [ ]:
#MS
#ETTh1:
Index:0,CRPS_L:1,AVE:0.18990368,var:0.0
Index:0,MSE_L:1,AVE:0.12117207,var:0.0
Index:0,MAE_L:1,AVE:0.2073799,var:0.0024236497
#ETTh2
Index:0,CRPS_L:1,AVE:0.2722967,var:0.0
Index:0,MSE_L:1,AVE:0.24472795,var:0.0
Index:0,MAE_L:1,AVE:0.28396863,var:0.007662289
#Weather
Index:0,CRPS_L:1,AVE:0.18372725,var:0.0
Index:0,MSE_L:1,AVE:0.1277009,var:0.0
Index:0,MAE_L:1,AVE:0.19873646,var:0.0029750222

#WO REG
#Exchange
Index:7,CRPS_L:8,AVE:0.31373262,var:0.0034177054
Index:7,MSE_L:8,AVE:0.37887412,var:0.190408
Index:7,MAE_L:8,AVE:0.32387847,var:0.0671021

#ETTh1
Index:0,CRPS_L:1,AVE:0.16855828,var:0.0
Index:0,MSE_L:1,AVE:0.09799735,var:0.0
Index:0,MAE_L:1,AVE:0.18251573,var:0.002681852
#ETTh2
Index:0,CRPS_L:1,AVE:0.26861325,var:0.0
Index:0,MSE_L:1,AVE:0.2357498,var:0.0
Index:0,MAE_L:1,AVE:0.27262577,var:0.010254114
#weather
Index:0,CRPS_L:1,AVE:0.19781663,var:0.0
Index:0,MSE_L:1,AVE:0.15007746,var:0.0
Index:0,MAE_L:1,AVE:0.22374451,var:0.0024221768

#_________________________________________________
#W REG
#Single
#Exchange
Index:7,CRPS_L:8,AVE:0.23455146,var:0.0013202557
Index:7,MSE_L:8,AVE:0.20312688,var:0.038856484
Index:7,MAE_L:8,AVE:0.25125167,var:0.024895169
#Elecrans
Index:0,CRPS_L:321,AVE:0.1714941,var:0.00011770314
Index:0,MSE_L:321,AVE:0.13653912,var:0.021596305
Index:0,MAE_L:321,AVE:0.16777086,var:0.010939249

#ETTh1
Index:0,CRPS_L:1,AVE:0.1713251,var:0.0
Index:0,MSE_L:1,AVE:0.101203896,var:0.0
Index:0,MAE_L:1,AVE:0.1860563,var:0.0026892824

#ETTh2
Index:0,CRPS_L:1,AVE:0.26525223,var:0.0
Index:0,MSE_L:1,AVE:0.23398851,var:0.0
Index:0,MAE_L:1,AVE:0.27045283,var:0.01107575

#Weather
Index:0,CRPS_L:1,AVE:0.19394289,var:0.0
Index:0,MSE_L:1,AVE:0.1439406,var:0.0
Index:0,MAE_L:1,AVE:0.21706055,var:0.0025334116

SyntaxError: invalid syntax (801630156.py, line 3)